In [1]:
#importanto os pacotes que serão utilizados no projeto
from requests import get
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import time
import re
import datetime

In [2]:
url = 'https://www.cobasi.com.br/c/gatos/racao/racao-seca?terms=/gatos/racao/racao-seca/7%40dot%405%20kg/Adulto/Super%20Premium%20Natural?map=c,c,c,specificationFilter_291,specificationFilter_297,specificationFilter_470&page=1'
response = get(url)

# usando os parâmetros do Beautifulsoup para transformar a página em um objeto
soup = BeautifulSoup(response.text, 'html.parser')

# criando objetos individuais para cada objeto (ração), para que possam ser analisadas em um lastro de repetição.
racao_containers= soup.find_all(class_= 'bNtpHK')

# how many hostels on the page
len(racao_containers)

21

In [3]:
#analisando a árvore do objeto para entender como extrair as informações necessárias
primeira_racao = racao_containers[0]

print(primeira_racao.prettify())

<div class="styles__ProductItem-sc-1ac06td-0 bNtpHK" data-testid="product-item">
 <a href="/racao-farmina-n-d-prime-gatos-adultos-castrados-frango-e-roma-3635251/p?idsku=635251">
  <div class="styles__ImageBlock-sc-1ac06td-1 eKuMsG">
   <img alt="Ração N&amp;amp;D Prime Gatos Adultos Castrados Frango e Romã" height="180" loading="lazy" src="https://cobasi.vteximg.com.br/arquivos/ids/211131-180-180/Racao-N-amp-D-Prime-Gatos-Adultos-Castrado-Frango.jpg?v=638122542419470000" width="180"/>
  </div>
  <div class="styles__InfoBox-sc-1ac06td-5 brVZsi">
   <div class="MediaMatch-sc-8d15xu-0 jKBjhQ">
    <div class="styles__Highlights-sc-1ac06td-7 ZOUgn">
    </div>
    <div class="styles__Highlights-sc-1ac06td-7 ZOUgn">
    </div>
   </div>
   <div class="MediaMatch-sc-8d15xu-0 kBZFEW">
    <div class="styles__Header-sc-1ac06td-2 cKLKQc">
     <div class="styles__BrandName-sc-1ac06td-3 bNiXcY">
      N&amp;D
     </div>
     <div class="styles__Rating-sc-1ac06td-12 kISiHB">
      <span>
      

In [4]:
# criando listas para cada valor que buscamos para montar a tabela final
racao_loja = []
racao_nome = []
racao_marca = []
racao_nota = []
racao_preco = []
racao_peso = []
data_acesso = []

#criando um lastro for para pegar cada informação de cada objeto e colocar nas listas criadas anteriormente
for item in range(len(racao_containers)):
    racao_loja.append(re.findall('www.(.*).com', url))
    racao_nome.append(racao_containers[item].find(class_ = 'dPsqyZ').get_text())
    racao_marca.append(racao_containers[item].find(class_ = 'bNiXcY').get_text())
    racao_nota.append(racao_containers[item].find(class_= 'kISiHB').get_text())
    if racao_containers[item].find(class_ = 'card-price') is not None:
       racao_preco.append(racao_containers[item].find(class_ = 'card-price').get_text()[3:].replace(',', '.'))
    else:
       racao_preco.append('na')
    if racao_containers[item].find(class_ = 'bOmFEM') is not None:
       racao_peso.append(racao_containers[item].find(class_= 'bOmFEM').get_text())
    else:
       racao_peso.append('na')
    data_acesso.append(datetime.date.today().strftime('%Y-%m-%d'))
time.sleep(2) # tempo para carregar cada vez e não sobrecarregar o server

In [7]:
#criando a tabela no Pandas
racao_gatos = pd.DataFrame({
    'racao_loja': racao_loja,
    'racao_nome': racao_nome,
    'racao_marca': racao_marca,
    'racao_nota': racao_nota,
    'racao_preco': racao_preco,
    'racao_peso': racao_peso,
    'data_acesso': data_acesso
})

#vendo o resultado final
racao_gatos.head()

,racao_loja,racao_nome,racao_marca,racao_nota,racao_preco,racao_peso,data_acesso
0,[cobasi],Ração N&D Prime Gatos Adultos Castrados Frango...,N&D,4.9,49.58,400 g,2024-01-29
1,[cobasi],Ração Biofresh Gatos Castrados Salmão,Biofresh,4.9,46.90,400 g,2024-01-29
2,[cobasi],Ração Premier Nattu Gatos Adultos Castrados Ab...,Premier,4.8,79.90,"1,5 kg",2024-01-29
3,[cobasi],Ração Guabi Natural Gato Adulto Castrado Frang...,Guabi Natural,4.8,92.90,"1,5 kg",2024-01-29
4,[cobasi],Ração N&D Ancestral Grain Gatos Adultos Castrados,N&D,4.9,49.00,400 g,2024-01-29


In [6]:
#criando arquivo csv do resultado
racao_gatos.to_csv('racao_gatos.csv', index=False)